In [18]:

%pip install pandas numpy matplotlib

Note: you may need to restart the kernel to use updated packages.


In [19]:

# TODO: Importera pandas
import pandas as pd

!pwd
%cd "../data"
# TODO: Ladda in data från CSV-fil
df = pd.read_csv('winequality-red.csv')

# TODO: Visa första 5 raderna
df.head()

# TODO: Visa antal rader och kolumner
df.shape

# TODO: Lista alla kolumner
df.columns


/workspaces/Big_data_py/tst/data
/workspaces/Big_data_py/tst/data


Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [ ]:
# TODO: Kolla efter saknade värden


# TODO: Kolla efter duplikater


# TODO: Visa statistik för numeriska kolumner


# TODO: Kolla efter outliers (extremvärden)
df_test = df.copy()

# print(f"total rows: {len(df)}")

for col in df_test.columns:
    Q1 = df_test[f"{col}"].quantile(0.25)
    Q3 = df_test[f"{col}"].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    print(f"col: {col} --- IQR: {IQR} --- low: {lower_bound} --- high: {upper_bound}")

    outlier = df_test[(df_test[f"{col}"] > lower_bound) | (df_test[f"{col}"] < upper_bound)]
    # print("Outliers:\n")
outlier

col: fixed acidity --- IQR: 2.0999999999999996 --- low: 3.95 --- high: 12.349999999999998
col: volatile acidity --- IQR: 0.25 --- low: 0.015000000000000013 --- high: 1.0150000000000001
col: citric acid --- IQR: 0.32999999999999996 --- low: -0.4049999999999999 --- high: 0.9149999999999999
col: residual sugar --- IQR: 0.7000000000000002 --- low: 0.8499999999999996 --- high: 3.6500000000000004
col: chlorides --- IQR: 0.01999999999999999 --- low: 0.04000000000000002 --- high: 0.11999999999999998
col: free sulfur dioxide --- IQR: 14.0 --- low: -14.0 --- high: 42.0
col: total sulfur dioxide --- IQR: 40.0 --- low: -38.0 --- high: 122.0
col: density --- IQR: 0.002234999999999987 --- low: 0.9922475000000001 --- high: 1.0011875
col: pH --- IQR: 0.18999999999999995 --- low: 2.925 --- high: 3.6849999999999996
col: sulphates --- IQR: 0.17999999999999994 --- low: 0.28000000000000014 --- high: 0.9999999999999999
col: alcohol --- IQR: 1.5999999999999996 --- low: 7.1000000000000005 --- high: 13.5
col: 

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5
